In [1]:
import $ivy.`org.apache.spark::spark-sql:3.1.1`
import $ivy.`org.apache.spark::spark-mllib:3.1.1`
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}

import $ivy.$                                  

import $ivy.$                                    

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

In [2]:
Logger.getLogger("org").setLevel(Level.WARN)

In [3]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
//	.config("sspark.executor..instances","2")
//	.config("spark.executor.memory","2G")
    .getOrCreate()
}

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/09/25 15:43:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@39ab2f9f

### The following class will help display Dataframes 

In [4]:
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit:Int = 20, truncate: Int = 20) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map { cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }

        publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map { row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)        
    }
}

defined class RichDF

In [5]:
import spark.implicits._
// in Scala
  case class Flight(DEST_COUNTRY_NAME: String,
                    ORIGIN_COUNTRY_NAME: String,
                    count: BigInt)
  val flightsDF = spark.read
					   .parquet("/Users/mehdi/Documents/Spark-The-Definitive-Guide/data/flight-data/parquet/2010-summary.parquet/")
  val flights = flightsDF.as[Flight]

parquet at cmd4.sc:7

1 / 1

import spark.implicits._
// in Scala
  
defined class Flight
flightsDF: DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]
flights: Dataset[Flight] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [6]:
flights.toDF.showHTML()

take at cmd3.sc:4

1 / 1

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,1
United States,Ireland,264
United States,India,69
Egypt,United States,24
Equatorial Guinea,United States,1
United States,Singapore,25
United States,Grenada,54
Costa Rica,United States,477
Senegal,United States,29
United States,Marshall Islands,44


In [7]:
// in Scala
flights.filter(flight_row => flight_row.DEST_COUNTRY_NAME == "United Kingdom")
	   .map(flight_row => flight_row)
	//    .take(5)
	   .toDF
	   .showHTML()

take at cmd3.sc:4

1 / 1

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United Kingdom,United States,1629


In [8]:
flights.filter(flight_row => flight_row.ORIGIN_COUNTRY_NAME != "Canada")
       .map(fr => Flight(fr.DEST_COUNTRY_NAME, fr.ORIGIN_COUNTRY_NAME, fr.count + 5))
	   .toDF
	   .showHTML()

take at cmd3.sc:4

1 / 1

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,6
United States,Ireland,269
United States,India,74
Egypt,United States,29
Equatorial Guinea,United States,6
United States,Singapore,30
United States,Grenada,59
Costa Rica,United States,482
Senegal,United States,34
United States,Marshall Islands,49


In [8]:
//spark.conf.set("spark.sql.shuffle.partitions", "20")

In [9]:
// in Scala
val staticDataFrame = spark.read
						   .format("csv")
						   .option("header", "true")
						   .option("inferSchema", "true")
						   .load("/Users/mehdi/Documents/Spark-The-Definitive-Guide/data/retail-data/by-day/*.csv")

load at cmd8.sc:5

305 / 305

load at cmd8.sc:5

305 / 305

load at cmd8.sc:5

1 / 1

load at cmd8.sc:5

12 / 12

staticDataFrame: DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]

In [10]:
staticDataFrame.createOrReplaceTempView("retail_data")
val staticSchema = staticDataFrame.schema

staticSchema: types.StructType = StructType(
  StructField("InvoiceNo", StringType, true, {}),
  StructField("StockCode", StringType, true, {}),
  StructField("Description", StringType, true, {}),
  StructField("Quantity", IntegerType, true, {}),
  StructField("InvoiceDate", StringType, true, {}),
  StructField("UnitPrice", DoubleType, true, {}),
  StructField("CustomerID", DoubleType, true, {}),
  StructField("Country", StringType, true, {})
)

In [11]:
// in Scala
import org.apache.spark.sql.functions.{window, column, desc, col}
staticDataFrame
.selectExpr(
	"CustomerId",
	"(UnitPrice * Quantity) as total_cost",
	"InvoiceDate")
.groupBy(
	col("CustomerId"), window(col("InvoiceDate"), "1 day"))
.sum("total_cost")
.showHTML()

take at cmd3.sc:4

12 / 12

take at cmd3.sc:4

1 / 1

CustomerId,window,sum(total_cost)
16057.0,[2011-12-04 17:00...,-37.6
14126.0,[2011-11-28 17:00...,643.6300000000001
13500.0,[2011-11-15 17:00...,497.9700000000001
17160.0,[2011-11-07 17:00...,516.8499999999999
15608.0,[2011-11-10 17:00...,122.4
15253.0,[2011-11-22 17:00...,277.6
15124.0,[2011-11-16 17:00...,93.44
12539.0,[2011-11-16 17:00...,1050.66
13658.0,[2011-11-29 17:00...,542.4000000000001
17396.0,[2011-10-30 17:00...,495.0


import org.apache.spark.sql.functions.{window, column, desc, col}


In [12]:
val streamingDataFrame = spark.readStream
      .schema(staticSchema)
      .option("maxFilesPerTrigger", 1)
      .format("csv")
      .option("header", "true")
      .load("/data/retail-data/by-day/*.csv")

streamingDataFrame: DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]

In [13]:
streamingDataFrame.isStreaming //return true

res12: Boolean = true

In [14]:
// in Scala
  val purchaseByCustomerPerHour = streamingDataFrame
    .selectExpr(
      "CustomerId",
      "(UnitPrice * Quantity) as total_cost",
      "InvoiceDate")
    .groupBy(
      $"CustomerId", window($"InvoiceDate", "1 day"))
    .sum("total_cost")

purchaseByCustomerPerHour: DataFrame = [CustomerId: double, window: struct<start: timestamp, end: timestamp> ... 1 more field]

In [15]:
// in Scala
purchaseByCustomerPerHour.writeStream
	.format("memory") // memory = store in-memory table
	.queryName("customer_purchases") // the name of the in-memory table
	.outputMode("complete") // complete = all the counts should be in the table
	.start()

21/09/25 16:31:54 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/pm/y4lv5tyd1rzfr9sx2jzdn2jw0000gn/T/temporary-de73c58a-0fb2-4a8d-b43c-0c5df2e6741a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


res14: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@61ffc80

In [16]:
// in Scala
spark.sql(s"""
SELECT *
FROM customer_purchases
ORDER BY `sum(total_cost)` DESC
""")
.show(10)

+----------+------+---------------+
|CustomerId|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+



In [17]:
purchaseByCustomerPerHour.writeStream
      .format("console")
      .queryName("customer_purchases_2")
      .outputMode("complete")
.start()

21/09/25 16:32:19 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/pm/y4lv5tyd1rzfr9sx2jzdn2jw0000gn/T/temporary-0dca8bc0-80e1-40c1-b642-659bab6546ce. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


res16: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1c8825b7

In [18]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [19]:
// in Scala
import org.apache.spark.sql.functions.date_format
val preppedDataFrame = staticDataFrame.na
									  .fill(0)
									  .withColumn("day_of_week", date_format($"InvoiceDate", "EEEE"))
									  .coalesce(5)

import org.apache.spark.sql.functions.date_format

preppedDataFrame: Dataset[Row] = [InvoiceNo: string, StockCode: string ... 7 more fields]

In [20]:
// in Scala
val trainDataFrame = preppedDataFrame.where("InvoiceDate < '2011-07-01'")
val testDataFrame = preppedDataFrame.where("InvoiceDate >= '2011-07-01'")

trainDataFrame: Dataset[Row] = [InvoiceNo: string, StockCode: string ... 7 more fields]
testDataFrame: Dataset[Row] = [InvoiceNo: string, StockCode: string ... 7 more fields]

In [21]:
trainDataFrame.count()
testDataFrame.count()

count at cmd20.sc:1

5 / 5

count at cmd20.sc:1

1 / 1

count at cmd20.sc:2

5 / 5

count at cmd20.sc:2

1 / 1

res20_0: Long = 245903L
res20_1: Long = 296006L

In [22]:
// in Scala
import org.apache.spark.ml.feature.StringIndexer
val indexer = new StringIndexer().setInputCol("day_of_week")
								 .setOutputCol("day_of_week_index")

import org.apache.spark.ml.feature.StringIndexer

indexer: StringIndexer = strIdx_4f81538457f7

In [23]:
// in Scala
import org.apache.spark.ml.feature.OneHotEncoder
val encoder = new OneHotEncoder()
.setInputCol("day_of_week_index")
.setOutputCol("day_of_week_encoded")

import org.apache.spark.ml.feature.OneHotEncoder

encoder: OneHotEncoder = oneHotEncoder_276ad580767a

In [24]:
// in Scala
  import org.apache.spark.ml.feature.VectorAssembler
  val vectorAssembler = new VectorAssembler()
    .setInputCols(Array("UnitPrice", "Quantity", "day_of_week_encoded"))
    .setOutputCol("features")

import org.apache.spark.ml.feature.VectorAssembler
  
vectorAssembler: VectorAssembler = VectorAssembler: uid=vecAssembler_9f8fd2ab74d2, handleInvalid=error, numInputCols=3

In [25]:

// in Scala
  import org.apache.spark.ml.Pipeline
  val transformationPipeline = new Pipeline()
    .setStages(Array(indexer, encoder, vectorAssembler))

import org.apache.spark.ml.Pipeline
  
transformationPipeline: Pipeline = pipeline_f1a21f252baf

In [26]:

// in Scala
  val fittedPipeline = transformationPipeline.fit(trainDataFrame)

collect at StringIndexer.scala:204

5 / 5

collect at StringIndexer.scala:204

1 / 1

fittedPipeline: org.apache.spark.ml.PipelineModel = pipeline_f1a21f252baf

In [27]:
// in Scala
  val transformedTraining = fittedPipeline.transform(trainDataFrame)

transformedTraining: DataFrame = [InvoiceNo: string, StockCode: string ... 10 more fields]

In [28]:

transformedTraining.cache()

res27: DataFrame = [InvoiceNo: string, StockCode: string ... 10 more fields]

In [29]:
// in Scala
  import org.apache.spark.ml.clustering.KMeans
  val kmeans = new KMeans()
    .setK(20)
    .setSeed(1L)

import org.apache.spark.ml.clustering.KMeans
  
kmeans: KMeans = kmeans_3363f4d08508

In [30]:
// in Scala
  val kmModel = kmeans.fit(transformedTraining)

takeSample at KMeans.scala:377

5 / 5

takeSample at KMeans.scala:377

5 / 5

sum at KMeans.scala:397

...

collect at ClusteringSummary.scala:49

5 / 5

collect at ClusteringSummary.scala:49

200 / 200

kmModel: org.apache.spark.ml.clustering.KMeansModel = KMeansModel: uid=kmeans_3363f4d08508, k=20, distanceMeasure=euclidean, numFeatures=7

In [31]:
spark.sparkContext.parallelize(Seq(1, 2, 3)).toDF()

res30: DataFrame = [value: int]